In [4]:
# %pip install tensorflow==2.6.2

In [29]:
# Importing libraries

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import numpy as np
import scipy.io
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import keras.backend as K
import numpy as np
import os
import random
import seaborn as sns
from datetime import datetime
from random import randrange
import matplotlib.pyplot as plt
import math
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import FileLink, FileLinks
import warnings
import pprint
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(42)

2.6.2


In [6]:
# Data loading
mat = scipy.io.loadmat('/home/nicolo/Scrivania/mat/pydata.mat')
data = mat["data"]

# Data correction
temp = abs(data) > 3
data[temp] = data[temp]/1000
data.shape

(47252, 10)

In [7]:
# Data preprocessing
psm1 = data[:,[1,2,3]].astype('float64')
psm2 = data[:,[4,5,6]].astype('float64')

cam = data[:,0].astype('int')
psm_features = data[:,[1,2,3,4,5,6]].astype('float64')

# psm1x = data[:,1].astype('float64')
# psm1y = data[:,2].astype('float64')
# psm1z = data[:,3].astype('float64')
# psm2x = data[:,4].astype('float64')
# psm2y = data[:,5].astype('float64')
# psm2z = data[:,6].astype('float64')

ecmx = data[:,7].astype('float64')
ecmy = data[:,8].astype('float64')
ecmz = data[:,9].astype('float64')

dist = np.sqrt(np.sum((psm1-psm2)**2, axis=1))

In [8]:
# Cam = 2 signal elimination
loc = cam==2
cam[loc] = 0
np.sum(cam)

4323

In [9]:
# Clutch vector creation
difference = cam[1:]-cam[:-1]

press = np.where(difference==1)[0][:]
release_temp = np.where(difference==-1)[0][:]
release = release_temp[1:]

clutch_pressed = difference == 1
clutch_released = difference == -1

clutch = np.zeros(cam.shape[0]-1)
clutch[clutch_pressed] = 1
clutch = np.insert(clutch, 0, 0, axis=0)

print("Camera pedal has been pressed n times: ", press.shape[0])
print("Camera pedal has been released n times: ", release.shape[0])
print("First press at: ", press[0])
print("First release at: ", release[0])

Camera pedal has been pressed n times:  119
Camera pedal has been released n times:  119
First press at:  61
First release at:  101


In [10]:
# Selection of usable windows
len=30
n_variables = 7
telescope = 3
samples = cam.shape[0]

p = press[1:]
r = release[:-1]

usable = np.where(clutch == 1)[0][:]
unusable = np.where((p-r)<len+telescope)[0][:]
clutch[press[unusable]+1] = 0
print("Usable clutches: ", np.sum(clutch))

Usable clutches:  112.0


In [11]:
# Empty matrices creation
distance=np.expand_dims(dist, axis=1)

windows = np.zeros((cam.shape[0]-len-telescope,len,n_variables))
features = np.concatenate((psm_features,distance), axis=1)
print("Windows matrix dimensions: ", windows.shape)
print("Features matrix dimensions: ", features.shape)

label_vector = np.zeros((samples-len-telescope,1))
label_matrix_temp = np.zeros((samples-len-telescope,n_variables))

# If the starting clutch is before len+telescope, we cannot label it
index = np.where(clutch==1)
print("First camera movement at timestamp: ", index[0][0])

Windows matrix dimensions:  (47219, 30, 7)
Features matrix dimensions:  (47252, 7)
First camera movement at timestamp:  62


In [12]:
# Useful matrices creation
label_vector[usable-len-telescope] = 1;
print('Matrix containing camera windows labels at each timestamp: ', label_vector.shape)
# label_matrix_temp[usable-len-telescope] = 1;

for j in range(n_variables):
    for i in range(samples-len-telescope):
        temp = features[i:i+len,j]
        windows[i,:,j] = temp

print("Matrix containing kinematic features at each timestamp: ", windows.shape)

Matrix containing camera windows labels at each timestamp:  (47219, 1)
Matrix containing kinematic features at each timestamp:  (47219, 30, 7)


In [13]:
# Stratified train-test split
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(windows, label_vector, test_size=0.25, stratify=label_vector)

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (35414, 30, 7)
y_train shape:  (35414, 1)
X_test shape:  (11805, 30, 7)
y_test shape:  (11805, 1)


In [14]:
# Shapes
input_shape = windows.shape[1:]
output_shape = label_vector.shape[1:]
print('Input shape: ', input_shape)
print('Output shape: ', output_shape)

Input shape:  (30, 7)
Output shape:  (1,)


In [15]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(64, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.3)(convlstm)

    # In order to predict the next values for more than one sensor,
    # we can use a Dense layer with a number given by telescope*num_sensors,
    # followed by a Reshape layer to obtain a tensor of dimension 
    # [None, telescope, num_sensors]
    output_layer = tfkl.Dense(output_shape[-1]*1, activation='sigmoid')(convlstm)
    #output_layer = tfkl.Reshape((1,output_shape[-1]))(output_layer)
    #output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='cnn_lstm_model')

    # Compile the model
    model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(lr = 1e-3),
                  metrics=tf.keras.metrics.Accuracy())
    # Return the model
    return model


model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

2022-03-29 11:53:21.113663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-29 11:53:21.113686: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-29 11:53:21.113705: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nicolo-Inspiron-14-5410): /proc/driver/nvidia/version does not exist
2022-03-29 11:53:21.113968: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "cnn_lstm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 30, 7)]           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 30, 128)           36864     
_________________________________________________________________
conv1d (Conv1D)              (None, 30, 64)            24640     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 15, 64)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 15, 128)           66048     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15, 128)           49280     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)            

In [16]:
import numpy as np
import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune 

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weights = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weights[key] = score if score > 1.0 else 1.0
    
    return class_weights

labels_dict = {0.0: 47107,
               1.0: 112}

class_weights = create_class_weight(labels_dict)
pprint.pprint(class_weights)

{0.0: 1.0, 1.0: 4.146932776767107}


In [27]:
y_train[:,0].shape

(35414,)

In [33]:
# Another option from scikit-learn 
from sklearn.utils import class_weight
import numpy as np

cls_weights = class_weight.compute_class_weight('balanced', np.unique(y_train[:,0]), y_train[:,0])
cls_weight_dict = {0: cls_weights[0], 1: cls_weights[1]}
val_sample_weights = class_weight.compute_sample_weight(cls_weight_dict, y_test)

print(val_sample_weights)
pprint.pprint(cls_weight_dict)

[0.50125973 0.50125973 0.50125973 ... 0.50125973 0.50125973 0.50125973]
{0: 0.5012597310686483, 1: 198.95505617977528}


In [ ]:
# class_weights = {0.0: 112/47219,
#                  1.0: 47107/47219}
# pprint.pprint(class_weights)

In [31]:
batch_size = 526
epochs = 100

history = model.fit(
    X_train,
    y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_test,  y_test),
    class_weight = class_weights,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='min', patience=20, 
                                    restore_best_weights=True, verbose = 1),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3,
                                        factor=0.1, min_lr=1e-7, verbose = 1)
    ]
).history

2022-03-29 12:03:38.657160: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
68/68 [==============================] - 17s 183ms/step - loss: 0.6989 - accuracy: 0.0000e+00 - val_loss: 0.6072 - val_accuracy: 0.0000e+00
Epoch 2/100
68/68 [==============================] - 11s 165ms/step - loss: 0.6971 - accuracy: 0.0000e+00 - val_loss: 0.7705 - val_accuracy: 0.0000e+00
Epoch 3/100
68/68 [==============================] - 11s 166ms/step - loss: 0.7036 - accuracy: 0.0000e+00 - val_loss: 0.6874 - val_accuracy: 0.0000e+00
Epoch 4/100
68/68 [==============================] - 11s 165ms/step - loss: 0.6947 - accuracy: 0.0000e+00 - val_loss: 0.6986 - val_accuracy: 0.0000e+00

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/100
68/68 [==============================] - 11s 165ms/step - loss: 0.6934 - accuracy: 0.0000e+00 - val_loss: 0.6971 - val_accuracy: 0.0000e+00
Epoch 6/100
68/68 [==============================] - 11s 165ms/step - loss: 0.6929 - accuracy: 0.0000e+00 - val_loss: 0.6977 - val_accuracy: 0.0000e+00
Epoch 

In [ ]:
# df2 = df2[["correlation_Pwave80_lead1", "lag_Pwave80_lead1",
#        "lag_PRintervall80_lead1", "msd_PRintervall80_lead1",
#        "lag_beat80_lead1", "correlation_Pwave20_lead1", "lag_QRSwave20_lead1",
#        "lag_Pwave4_lead1", "lag_QRSwave4_lead1",
#        "correlation_PRintervall4_lead1", "lag_PRintervall4_lead1",
#        "values_under_mean_perc_Pwave_lead1", "mean_QRScomplex_lead1",
#        "median_QRScomplex_lead1", "values_over_mean_perc_QRScomplex_lead1",
#        "max_PRintervall_lead1", "min_PRintervall_lead1",
#        "kurtosis_PRintervall_lead1",
#        "values_under_mean_perc_PRintervall_lead1", "max_beat_lead1",
#        "min_beat_lead1", "values_under_mean_perc_beat_lead1",
#        "correlation_Pwave80_lead2", "lag_Pwave80_lead2",
#        "lag_PRintervall80_lead2", "lag_beat80_lead2",
#        "correlation_Pwave20_lead2", "lag_QRSwave20_lead2", "msd_beat20_lead2",
#        "lag_Pwave4_lead2", "lag_QRSwave4_lead2",
#        "correlation_PRintervall4_lead2", "lag_PRintervall4_lead2",
#        "lag_beat4_lead2", "values_under_mean_perc_Pwave_lead2",
#        "mean_QRScomplex_lead2", "median_QRScomplex_lead2",
#        "der_std_QRScomplex_lead2", "max_PRintervall_lead2",
#        "min_PRintervall_lead2", "kurtosis_PRintervall_lead2",
#        "values_under_mean_perc_PRintervall_lead2",
#        "max_min_diff_PRintervall_lead2", "max_beat_lead2", "min_beat_lead2",
#        "values_under_mean_perc_beat_lead2", "mean_rr_30s_back",
#        "std_drr_30s_back", "std_drr_60s_back", "mean_rr_120s_current",
#        "pNN50_120s_current", "rmssd_120s_current","beats","frequency"]]

In [ ]:
# df_dict = {}
# for i in range(320):
#     df_dict[f"df_predictions_{i}"] = pd.DataFrame(columns = ["predictions","predictions_letter"])
    
# for i in range(320):
#     matrix = df2["beats" == i]
#     if matrix["frequency"][1] == 250
#         prediction = model1.predict(X1)
#     else
#         prediction = model2.predict(X2)
    
#     df_dict[f"df_predictions_{i}"]["predictions"] = predictions
#     df_dict[f"df_predictions_{i}"]["predictions_letter"] = 'N' if df_dict[f"df_predictions_{i}"]["predictions_letter"] == 0 else  'S' if f_dict[f"df_predictions_{i}"]["predictions_letter"] == 1 else 'V'
    
# #     DF250.to_csv("tabella250.csv", index=False)
# #     DF128.to_csv("Satisfaction.csv", index=False)

In [ ]:
# DF250 = pd.DataFrame(prediction250)
# DF128 = pd.DataFrame(prediction128)
# DF250.to_csv("tabella250.csv", index=False)
# DF128.to_csv("Satisfaction.csv", index=False)